In [ ]:
#pip install transformers
#pip install sentence_transformers

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel, GPT2Model, CLIPTextModel, ElectraModel
import os
import tqdm

2023-09-26 18:47:26.391981: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 18:47:27.194531: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
'''
from sentence_transformers import SentenceTransformer, util
model_clip = SentenceTransformer('clip-ViT-B-32')
text_embedding_clip = model_clip.encode(text)
'''

In [2]:
def files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file

In [3]:
def text_embeding(text, model, tokenizer):
    inputs = tokenizer([text], return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    embeddings = embeddings.tolist()[0]
    return embeddings


In [4]:
distilbert = "distilbert-base-multilingual-cased"
tokenizer_distilbert = AutoTokenizer.from_pretrained(distilbert)
model_distilbert = AutoModel.from_pretrained(distilbert)

minilm = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer_minilm = AutoTokenizer.from_pretrained(minilm)
model_minilm = AutoModel.from_pretrained(minilm)

xtremedistil = "microsoft/xtremedistil-l6-h256-uncased"
tokenizer_xtremedistil = AutoTokenizer.from_pretrained(xtremedistil)
model_xtremedistil = AutoModel.from_pretrained(xtremedistil)

gpt2 = "gpt2"
tokenizer_gpt2 = AutoTokenizer.from_pretrained(gpt2)
model_gpt2 = GPT2Model.from_pretrained(gpt2)

electra = "google/electra-small-discriminator"
tokenizer_electra = AutoTokenizer.from_pretrained(electra)
model_electra = ElectraModel.from_pretrained(electra)

clip = "openai/clip-vit-base-patch32"
tokenizer_clip = AutoTokenizer.from_pretrained(clip)
model_clip = CLIPTextModel.from_pretrained(clip)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', '

In [ ]:
'''
train = pd.read_csv('./coco/annotations/train_annotations.csv')
text = train['caption'].iloc[0]
text_embeding(text, model_clip, tokenizer_clip)
'''

In [10]:
train = pd.read_csv('./coco/annotations/train_annotations.csv')

In [11]:
# отбираем картинки, для которых были созданы эмбединги

img_encoding_path = './img_encodings'
emdeded_img = []

for f in files(path = img_encoding_path):
    path = "./img_encodings/" + f
    df = pd.read_parquet(path)
    imgs = list(df['img'])
    emdeded_img.append(imgs)


emdeded_img = sum(emdeded_img, [])

emdeded_img = list(set(emdeded_img))

train['embeded_img'] = train['image_id'].apply(lambda x: x in emdeded_img)

train = train[train.embeded_img == True]

In [14]:
def get_text_encoding_file(model,model_name,tokenizer,count_img):
    i = 0
    file_num = 0

    while i<=len(train):

        list_img = []
        list_ids = []
        list_encodings = []

        df = train.iloc[i:i+count_img]
        for j in tqdm.tqdm(range(len(df))):
            img = df['image_id'].iloc[j]
            id = df['id'].iloc[j]
            text = df['caption'].iloc[j]

            list_img.append(img)
            list_ids.append(id)
            list_encodings.append(text_embeding(text, model, tokenizer))

        df = pd.DataFrame(list_encodings)
        df['img'] = list_img
        df['id'] = list_ids
        df.columns = df.columns.map(str)

        file_name = "./text_encodings/train_text_encodings_" + f"{model_name}" + "_" + f"{file_num}" + ".parquet"

        df.to_parquet(file_name)  

        file_num = file_num + 1
        i = i + count_img

In [ ]:
model = model_clip
model_name = 'clip'
tokenizer = tokenizer_clip
count_img = 10

get_text_encoding_file(model,model_name,tokenizer,count_img)

In [ ]:
'''
restored_df = pd.read_parquet(path = './text_encodings/train_text_encodings_clip_28.parquet')
restored_df
'''